In [1]:
import nltk

from gensim.models import Word2Vec
from nltk.corpus import stopwords

import re

In [2]:
# Download Punkt tokenizer (if not already downloaded)
try:
  nltk.data.find('tokenizers/punkt/PY3/english.pickle')
except LookupError:
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
paragraph = """I have three visions for India. In 3000 years of our history, people from all over
               the world have come and invaded us, captured our lands, conquered our minds.
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours.
               Yet we have not done this to any other nation. We have not conquered anyone.
               We have not grabbed their land, their culture,
               their history and tried to enforce our way of life on them.
               Why? Because we respect the freedom of others.That is why my
               first vision is that of freedom. I believe that India got its first vision of
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India
               stands up to the world, no one will respect us. Only strength respects strength. We must be
               strong not only as a military power but also as an economic power. Both must go hand-in-hand.
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life.
               I see four milestones in my career"""


In [5]:
# Preprocessing the data
text = re.sub(r'\[[0-9]*\]',' ',paragraph)
text = re.sub(r'\s+',' ',text)
text = text.lower()
text = re.sub(r'\d',' ',text)
text = re.sub(r'\s+',' ',text)

In [6]:
# Preparing the dataset
sentences = nltk.sent_tokenize(text)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

In [7]:
for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

In [9]:
# Training the Word2Vec model
model = Word2Vec(sentences, min_count=1)


word_index = model.wv.key_to_index['good']

In [10]:
word_index

73

In [15]:
def get_word_info(word):
  """
  Attempts to retrieve word index and vector from the model, handling potential errors.

  Args:
      word: The word to get information for.

  Returns:
      A tuple containing (word_index, word_vector) or (None, None) if errors occur.
  """
  try:
    word_index = model.wv.key_to_index.get(word)
    if word_index is not None:
      try:
        word_vector = model.wv.get_vecattr(word, 'vector')
        return word_index, word_vector
      except KeyError:  # Handle potential KeyError for missing vector attribute
        print(f"Word '{word}' has index {word_index} but vector attribute might not be available")
        return word_index, None
    else:
      print(f"Word '{word}' not found in vocabulary")
      return None, None
  except (AttributeError, KeyError) as e:  # Handle general errors related to model compatibility
    print(f"An error occurred while processing '{word}': {e}")
    return None, None

# Example usage
word = "military"
word_index, word_vector = get_word_info(word)
if word_vector is not None:
  print(f"Word vector for '{word}': {word_vector[:10]} (truncated)")

word = "war"
word_index, word_vector = get_word_info(word)
if word_vector is not None:
  print(f"Word vector for '{word}': {word_vector[:10]} (truncated)")

Word 'military' has index 68 but vector attribute might not be available
Word 'war' has index 31 but vector attribute might not be available


In [16]:
def find_similar_words(word, top_n=10):
  """
  Finds the most similar words to a given word using word2vec model.

  Args:
      word: The word to find similar words for.
      top_n: The number of most similar words to return (default: 10).

  Returns:
      A list of tuples containing (similar_word, similarity score).
  """
  try:
    similar_words = model.wv.most_similar(word, topn=top_n)
    return similar_words
  except KeyError:
    print(f"Word '{word}' not found in vocabulary")
    return []

In [20]:
word = "strength"
similar_words = find_similar_words(word)

if similar_words:
  for similar_word, similarity in similar_words:
    print(f"'{similar_word}': {similarity:.2f}")  # Print word and similarity score with 2 decimal places
else:
  print("No similar words found.")

'land': 0.23
'lucky': 0.22
'.': 0.22
'space': 0.20
'succeeded': 0.19
'looted': 0.18
'came': 0.18
'us': 0.18
'material': 0.17
'among': 0.17
